# Trabalho No. 3

- Aluno: Guilherme Lisboa de Oliveira (GRR20170414)

```{admonition} Questão 1

A Tabela 1 classifica 90 pacientes com câncer de mama de acordo com sua idade no momento da inscrição em um ensaio clínico e se elas haviam recebido ou não quimioterapia anterior. Pacientes que receberam quimioterapia anteriormente e, possivelmente falharam, têm uma resposta clínica diferente daqueles que nunca receberam quimioterapia.

| Idade    | Não quimioterapia | Sim quimioterapia | Total |
|:--------:|:-----------------:|:-----------------:|:-----:|
| $\lt$35  | 0                 | 14                | 14    |
| 35-39    | 2                 | 9                 | 11    |
| 40-44    | 11                | 12                | 23    |
| 45-49    | 21                | 13                | 34    |
| $\geq$50 | 8                 | 0                 | 8     |
| Total    | 42                | 48                | 90    |

Tabela 1: Pacientes com câncer de mama no estudo categorizados por idade e quimioterapia anterior.

a. Use a regressão logística para mostrar que a probabilidade de uso anterior de quimioterapia em um paciente com câncer inscrito neste ensaio clínico está relacionado à sua idade.

b. Explique essa descoberta e sugira dados adicionais que você precisa ver coletados para apoiar esta reivindicação.

```

**a.** 

In [21]:
import numpy as np
import pandas as pd

categorias_idade = ["<35", "35-39", "40-44", "45-49", ">=50"]
idades = [32, 37, 42, 47, 52]

q1_dados = pd.DataFrame(
    {
        "idade": pd.Categorical(categorias_idade, ordered=True, categories=categorias_idade),
        "não_quimio": [0, 2, 11, 21, 8],
        "sim_quimio": [14, 9, 12, 13, 0],
    }
)

q1_dados


,idade,não_quimio,sim_quimio
0,<35,0,14
1,35-39,2,9
2,40-44,11,12
3,45-49,21,13
4,>=50,8,0


Como o objetivo é definir a probabilidade de um paciente ter feito ou não quimioterapia, vamos expandir as observações resumidas na tabela de contigência para uma tabela completa dos dados, a coluna **resultado** indica se o paciente foi submetido ao tratamento. A coluna **categoria_idade** trata idade como dado categórico ordinal, já a coluna **idade** será utilizada como valor quantitativo. O objetivo é comparar a qualidade dos modelos que tem cada uma destas colunas como variável explicativa.

In [22]:
total_observacoes = q1_dados.sum(numeric_only=True).sum()
observacoes = np.empty([total_observacoes, 3], dtype=object)
offset = 0

for i in q1_dados.index:
    n_sucessos = q1_dados.loc[i]["sim_quimio"]
    n_tentativas = q1_dados.loc[i][1:3].sum()
    categoria_idade = q1_dados.loc[i]["idade"]
    idade = idades[i]

    observacoes[offset : offset + n_sucessos] = [1, categoria_idade, idade]
    observacoes[offset + n_sucessos : offset + n_tentativas] = [
        0,
        categoria_idade,
        idade,
    ]

    offset += n_tentativas


q1_dados_expandidos = pd.DataFrame(
    observacoes, columns=["resultado", "categoria_idade", "idade"]
)

q1_dados_expandidos["resultado"] = q1_dados_expandidos["resultado"].astype(np.int8)
q1_dados_expandidos["categoria_idade"] = q1_dados_expandidos["categoria_idade"].astype(
    q1_dados["idade"].dtype
)
q1_dados_expandidos["idade"] = q1_dados_expandidos["idade"].astype(np.int8)

q1_dados_expandidos


,resultado,categoria_idade,idade
0,1,<35,32
1,1,<35,32
2,1,<35,32
3,1,<35,32
4,1,<35,32
...,...,...,...
85,0,>=50,52
86,0,>=50,52
87,0,>=50,52
88,0,>=50,52


- Modelo resultado ~ idade:

In [23]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

modelo_idade = smf.glm(
    "resultado ~ idade",
    data=q1_dados_expandidos,
    family=sm.families.Binomial(),
)

fit_modelo_idade = modelo_idade.fit()

fit_modelo_idade.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
              Results: Generalized linear model
==============================================================
Model:              GLM              AIC:            96.4665  
Link Function:      Logit            BIC:            -303.5168
Dependent Variable: resultado        Log-Likelihood: -46.233  
Date:               2022-04-24 14:19 LL-Null:        -62.183  
No. Observations:   90               Deviance:       92.466   
Df Model:           1                Pearson chi2:   78.5     
Df Residuals:       88               Scale:          1.0000   
Method:             IRLS                                      
---------------------------------------------------------------
            Coef.   Std.Err.     z     P>|z|    [0.025   0.975]
---------------------------------------------------------------
Intercept  11.2502    2.5813   4.3584  0.0000   6.1910  16.3094
idade      -0.2570    0.0584  -4.3996  0.0000  -0.3715  -0.1425
==============================================================

"""

- Modelo resultado ~ categoria_idade:

In [24]:
modelo_categorico = smf.glm(
    "resultado ~ 0 + categoria_idade",
    data=q1_dados_expandidos,
    family=sm.families.Binomial(),
)

fit_modelo_categorico = modelo_categorico.fit()

fit_modelo_categorico.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                       Results: Generalized linear model
================================================================================
Model:                    GLM                    AIC:                  97.5062  
Link Function:            Logit                  BIC:                  -294.9776
Dependent Variable:       resultado              Log-Likelihood:       -43.753  
Date:                     2022-04-24 14:19       LL-Null:              -62.183  
No. Observations:         90                     Deviance:             87.506   
Df Model:                 4                      Pearson chi2:         68.0     
Df Residuals:             85                     Scale:                1.0000   
Method:                   IRLS                                                  
--------------------------------------------------------------------------------
                        Coef.    Std.Err.     z    P>|z|     [0.025     0.975]  
--------------------------------------------------------------------------------
categoria_idade[<35]    23.5661 21237.1184  0.0011 0.9991 -41600.4211 41647.5533
categoria_idade[35-39]   1.5041     0.7817  1.9240 0.0544     -0.0281     3.0363
categoria_idade[40-44]   0.0870     0.4174  0.2084 0.8349     -0.7311     0.9051
categoria_idade[45-49]  -0.4796     0.3529 -1.3589 0.1742     -1.1713     0.2121
categoria_idade[>=50]  -23.5661 28094.0613 -0.0008 0.9993 -55086.9144 55039.7822
================================================================================

"""

O modelo que utiliza a idade como dado categórico apresentou p-valores muito altos em todas as categorias e as categorias inicial e final, <35 e $\geq$50, tiveram erros muito acima das demais também. Como o modelo que considera idade como dado quantitativo apresentou um p-valor muito baixo e um AIC melhor, este foi escolhido.

Portanto, baseando-se no modelo escolhido, existe uma relação entre a idade do paciente e a probabilidade de ter realizado quimioterapia, sendo que quanto mais avançada a idade, menor a probabilidade de ter recebido o tratamento. Seguem abaixo os valores preditos para as idades usadas como variáveis quantitativas no ajuste do modelo:

In [25]:
pd.DataFrame({
    'Idade': idades,
    'p_predito': fit_modelo_idade.predict(exog=dict(idade=idades))
    })

,Idade,p_predito
0,32,0.953775
1,37,0.850944
2,42,0.612334
3,47,0.304120
4,52,0.107874


**b.** Como o modelo que uiliza idade como dado quantitativo apresentou melhores resultados, seria necessário coletar a idade dos pacientes desta forma também. Caso seja apenas possível receber os dados de forma categória, as erros altos na categorias inicial e final indicam que seria interessante expandi-las, buscando obter algo como idades <25, 25-30, 30-35 ... 50-55, 55-60, $\geq$60.

```{admonition} Questão 2

As águias procuram salmão, mas sabem que é mais fácil roubar de outra águia. A Tabela2 abaixo relaciona o número de tentativas de pirataria e o número de tentativas bem-sucedidas em um estudo de observação de biólogos no Estado de Washington. Os tamanhos do pirata e da vítima são codificados como L = grande, S = pequeno. A idade do pirata ́e codificada como A = adulto, I = imaturo.

| No. de sucesssos $y_i$ | No. de tentativas $n_i$ | Tamanho Pirata | Idade Pirata | Tamanho Vítima |
|:----------------------:|:-----------------------:|:--------------:|:------------:|:--------------:|
| 17                     | 24                      | L              | A            | L              |
| 29                     | 29                      | L              | A            | S              |
| 17                     | 27                      | L              | I            | L              |
| 20                     | 20                      | L              | I            | S              |
| 1                      | 12                      | S              | A            | L              |
| 15                     | 16                      | S              | A            | S              |
| 0                      | 28                      | S              | I            | L              |
| 1                      | 4                       | S              | I            | S              |

Tabela 2: Tentativas de pirataria de águia. O tamanho é codificado como grande e pequeno; idade é codificada como adulta ou imatura.

a. Use a regressão logística para mostrar que o tamanho do pirata, a idade e o tamanhoda vítima são fatores importantes para determinar se o pirata é bem-sucedido ou não. Mostre que o sinergismo da idade e do tamanho dos piratas também é importante.

b. Mostre que nenhuma das interações entre as variáveis podem ser medidas devido à separação no modelo logístico.

```

**a.** 

In [26]:
q2_dados = pd.DataFrame(
    {
        "y_i": [17, 29, 17, 20, 1, 15, 0, 1],
        "n_i": [24, 29, 27, 20, 12, 16, 28, 4],
        "tamanho_pirata": ["L", "L", "L", "L", "S", "S", "S", "S"],
        "idade_pirata": ["A", "A", "I", "I", "A", "A", "I", "I"],
        "tamanho_vitima": ["L", "S", "L", "S", "L", "S", "L", "S"],
    }
)

q2_dados


,y_i,n_i,tamanho_pirata,idade_pirata,tamanho_vitima
0,17,24,L,A,L
1,29,29,L,A,S
2,17,27,L,I,L
3,20,20,L,I,S
4,1,12,S,A,L
5,15,16,S,A,S
6,0,28,S,I,L
7,1,4,S,I,S


Assim como na questão anterior, vamos expandir a tabela e definir o **resultado** como uma variável dicotômica que indica se o pirata foi bem sucedido na observação.

In [27]:
observacoes = np.empty([q2_dados["n_i"].sum(), 4], dtype=object)
offset = 0

for i in q2_dados.index:
    n_sucessos, n_tentativas = q2_dados.loc[i][["y_i", "n_i"]]
    categorias = [*q2_dados.loc[i][2:5]]

    observacoes[offset : offset + n_sucessos] = [1, *categorias]
    observacoes[offset + n_sucessos : offset + n_tentativas] = [0, *categorias]

    offset += n_tentativas


q2_dados_expandidos = pd.DataFrame(
    observacoes, columns=["resultado", *q2_dados.columns[2:5]]
)

q2_dados_expandidos["resultado"] = q2_dados_expandidos["resultado"].astype(np.int8)

q2_dados_expandidos


,resultado,tamanho_pirata,idade_pirata,tamanho_vitima
0,1,L,A,L
1,1,L,A,L
2,1,L,A,L
3,1,L,A,L
4,1,L,A,L
...,...,...,...,...
155,0,S,I,L
156,1,S,I,S
157,0,S,I,S
158,0,S,I,S


- Modelo resultado ~ tamanho pirata + idade pirata + tamanho vitima

In [28]:
modelo_soma = smf.glm(
    "resultado ~ tamanho_pirata + idade_pirata + tamanho_vitima",
    data=q2_dados_expandidos,
    family=sm.families.Binomial(),
)

fit_modelo_soma = modelo_soma.fit()

fit_modelo_soma.summary2()


<class 'statsmodels.iolib.summary2.Summary'>
"""
                 Results: Generalized linear model
===================================================================
Model:                 GLM               AIC:             98.3892  
Link Function:         Logit             BIC:             -701.3380
Dependent Variable:    resultado         Log-Likelihood:  -45.195  
Date:                  2022-04-24 14:19  LL-Null:         -105.85  
No. Observations:      160               Deviance:        90.389   
Df Model:              3                 Pearson chi2:    97.6     
Df Residuals:          156               Scale:           1.0000   
Method:                IRLS                                        
-------------------------------------------------------------------
                     Coef.  Std.Err.    z    P>|z|   [0.025  0.975]
-------------------------------------------------------------------
Intercept            1.3416   0.4602  2.9151 0.0036  0.4396  2.2436
tamanho_pirata[T.S] -4.5570   1.0545 -4.3216 0.0000 -6.6236 -2.4903
idade_pirata[T.I]   -1.0973   0.5465 -2.0080 0.0446 -2.1684 -0.0262
tamanho_vitima[T.S]  4.9331   1.1193  4.4074 0.0000  2.7394  7.1269
===================================================================

"""

- Modelo resultado ~ tamanho pirata + idade pirata + tamanho vítima + interação tamanho e idade pirata

In [33]:
modelo_soma_e_interacao = smf.glm(
    "resultado ~ tamanho_pirata + idade_pirata + tamanho_vitima + tamanho_pirata:idade_pirata",
    data=q2_dados_expandidos,
    family=sm.families.Binomial(),
)

fit_modelo_soma_e_interacao = modelo_soma_e_interacao.fit()

fit_modelo_soma_e_interacao.summary2()


<class 'statsmodels.iolib.summary2.Summary'>
"""
                          Results: Generalized linear model
=====================================================================================
Model:                       GLM                     AIC:                   93.7656  
Link Function:               Logit                   BIC:                   -702.8863
Dependent Variable:          resultado               Log-Likelihood:        -41.883  
Date:                        2022-04-24 14:50        LL-Null:               -105.85  
No. Observations:            160                     Deviance:              83.766   
Df Model:                    4                       Pearson chi2:          87.7     
Df Residuals:                155                     Scale:                 1.0000   
Method:                      IRLS                                                    
-------------------------------------------------------------------------------------
                                       Coef.  Std.Err.    z    P>|z|   [0.025  0.975]
-------------------------------------------------------------------------------------
Intercept                              0.8977   0.4480  2.0040 0.0451  0.0197  1.7757
tamanho_pirata[T.S]                   -3.4605   1.1287 -3.0661 0.0022 -5.6727 -1.2484
idade_pirata[T.I]                     -0.3590   0.5986 -0.5997 0.5487 -1.5323  0.8143
tamanho_vitima[T.S]                    5.4324   1.3602  3.9938 0.0001  2.7665  8.0983
tamanho_pirata[T.S]:idade_pirata[T.I] -3.6614   1.6279 -2.2492 0.0245 -6.8521 -0.4708
=====================================================================================

"""

Comparando o modelo com e sem interação, vemos que o modelo que incluí a interação apresenta AIC e Deviance menores. Além disso, a interação tem baixo pvalor mostrando que há uma queda considerável na probabilidade de sucesso do pirata quando o mesmo é pequeno e imaturo.

Seguem abaixo valores preditos:

In [31]:
q2_dados["p_predito"] = fit_modelo_soma_e_interacao.predict(
    exog=dict(
        tamanho_pirata=q2_dados.tamanho_pirata,
        idade_pirata=q2_dados.idade_pirata,
        tamanho_vitima=q2_dados.tamanho_vitima,
    )
)

q2_dados


,y_i,n_i,tamanho_pirata,idade_pirata,tamanho_vitima,p_predito
0,17,24,L,A,L,0.710483
1,29,29,L,A,S,0.998221
2,17,27,L,I,L,0.631515
3,20,20,L,I,S,0.997455
4,1,12,S,A,L,0.071570
5,15,16,S,A,S,0.946322
6,0,28,S,I,L,0.001381
7,1,4,S,I,S,0.240330


**b.** 